In [5]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install folium
import folium
from folium import plugins
import ipywidgets
!pip install geocoder
import geocoder
!pip install geopy
import geopy
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
!pip install reverse_geocode
import reverse_geocode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Created wheel for reverse-geocode: filename=reverse_geocode-1.4.1-py3-none-any.whl size=1637841 sha256=f2317b6568b31bfbcffdde93e96fae8c8b61fde08dfb05dd57aada9b0fc9099f
  Stored in directory: /root/.cache/pip/wheels/34/fd/8c/204b58f530154e61f182bcf1813a53b3ad5844faaff3df51d6
Successfully built reverse-geocode


In [6]:
# importing data
df= pd.read_csv('/content/Hotel_Reviews.csv')
# changing column to lower case
df.columns=[x.lower() for x in df.columns]

In [7]:
#shape of dataframe
df.shape

(76527, 17)

In [8]:
# The dataset comprimises of 17 columns. 
df.columns

Index(['hotel_address', 'additional_number_of_scoring', 'review_date',
       'average_score', 'hotel_name', 'reviewer_nationality',
       'negative_review', 'review_total_negative_word_counts',
       'total_number_of_reviews', 'positive_review',
       'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'reviewer_score', 'tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [9]:
#there are 1492 hotels
hotel=list(df.hotel_name.unique())
len(hotel)

213

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76527 entries, 0 to 76526
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   hotel_address                               76527 non-null  object 
 1   additional_number_of_scoring                76526 non-null  float64
 2   review_date                                 76526 non-null  object 
 3   average_score                               76526 non-null  float64
 4   hotel_name                                  76526 non-null  object 
 5   reviewer_nationality                        76526 non-null  object 
 6   negative_review                             76526 non-null  object 
 7   review_total_negative_word_counts           76526 non-null  float64
 8   total_number_of_reviews                     76526 non-null  float64
 9   positive_review                             76526 non-null  object 
 10  review_tot

In [14]:
#impute the word "H tel" to "Hotel"
df['hotel_name']=df['hotel_name'].apply(lambda x:str(x).replace('H tel','Hotel'))

In [15]:
# sanity check to check
df[df.index==12608]['hotel_name']

12608    Hotel De Vend me
Name: hotel_name, dtype: object

In [16]:
#group by hotelname and aggregate
geocode_df = df.groupby('hotel_name').agg({'lat': 'first','lng':'first'}).reset_index()

In [17]:
#check if there is null
geocode_df.isnull().sum()

hotel_name    0
lat           1
lng           1
dtype: int64

In [18]:
#drop null and reset index
geocode_df.dropna(subset=['lat','lng'],inplace=True)
geocode_df.reset_index(drop='index',inplace=True)

In [19]:
#sanity check if there is null
geocode_df.isnull().sum()

hotel_name    0
lat           0
lng           0
dtype: int64

In [20]:
#create new column
geocode_df['location']= ''
#function to search for geocode
def search(x):
    for i in range (x.shape[0]):
        co= ((geocode_df['lat'][i], geocode_df['lng'][i]),)
        geocode_df['location'][i]= reverse_geocode.search(co)
#apply function
#search the df
search(geocode_df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [21]:
#function to search for country
def search_country(x):
    return x[0]['country']

In [22]:
#map the city
geocode_df['country']=geocode_df['location'].map(search_country)

In [23]:
geocode_df.country.unique()

array(['United Kingdom', 'France', 'Netherlands'], dtype=object)

In [24]:
#function to search for city
def search_city(x):
    return x[0]['city']

In [25]:
#map the city
geocode_df['city']=geocode_df['location'].map(search_city)

In [26]:
geocode_df.city.value_counts()

Paris                  57
Levallois-Perret       18
London                 17
West End of London     16
Kensington             14
Clichy                 14
Bayswater              10
Chelsea                 9
Vanves                  6
Saint-Ouen              6
Neuilly-sur-Seine       5
City of London          5
City of Westminster     4
Poplar                  4
Clerkenwell             3
Montrouge               3
Barbican                2
Belsize Park            2
Malakoff                2
Bethnal Green           2
Gentilly                2
Richmond                1
Camden Town             1
Ivry-sur-Seine          1
Barnsbury               1
Canary Wharf            1
Blackheath              1
Issy-les-Moulineaux     1
Lambeth                 1
Amsterdam               1
Le Kremlin-Bicêtre      1
Battersea               1
Name: city, dtype: int64

In [28]:
#save to pickle
geocode_df.to_pickle('geocode.pkl')

In [29]:
tag_df=df[['hotel_name','tags','lat','lng']]

In [30]:
tag_df.head()

,hotel_name,tags,lat,lng
0,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
1,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Duplex Double...",52.360576,4.915968
2,Hotel Arena,"[' Leisure trip ', ' Family with young childre...",52.360576,4.915968
3,Hotel Arena,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",52.360576,4.915968
4,Hotel Arena,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",52.360576,4.915968


In [32]:
tag_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76527 entries, 0 to 76526
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   hotel_name  76527 non-null  object 
 1   tags        76526 non-null  object 
 2   lat         76526 non-null  float64
 3   lng         76526 non-null  float64
dtypes: float64(2), object(2)
memory usage: 2.3+ MB


In [34]:
tag_df2 = tag_df

In [35]:
tag_df2['lat'] = tag_df2['lat'].astype(str)
tag_df2['lng'] = tag_df2['lng'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [36]:
tag_df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76527 entries, 0 to 76526
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hotel_name  76527 non-null  object
 1   tags        76526 non-null  object
 2   lat         76527 non-null  object
 3   lng         76527 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [42]:
tag_df2

In [41]:
tag_df2.shape

AttributeError: ignored

In [44]:
tag_df.shape

(76527, 4)

In [45]:
tag_df.isnull().sum()

hotel_name    0
tags          1
lat           0
lng           0
dtype: int64

In [46]:
tag_df.dropna(subset=['lat','lng'],inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [48]:
tag_df.to_pickle("hoteltag.pkl")

In [49]:
#suppose to be empty 
df[df['positive_review']=='No Positive'][['negative_review','positive_review']].head(5)

,negative_review,positive_review
8,Even though the pictures show very clean room...,No Positive
32,Our bathroom had an urine order Shower was ve...,No Positive
98,Got charged 50 for a birthday package when it...,No Positive
121,The first room had steep steps to a loft bed ...,No Positive
134,Foyer was a mess Only place to relax was the ...,No Positive


In [50]:
# suppose to be empty 

df[df['negative_review']=='No Negative'][['negative_review','positive_review']].head(5)

,negative_review,positive_review
1,No Negative,No real complaints the hotel was great great ...
13,No Negative,This hotel is being renovated with great care...
15,No Negative,This hotel is awesome I took it sincirely bec...
18,No Negative,Public areas are lovely and the room was nice...
48,No Negative,The quality of the hotel was brilliant and ev...


In [52]:
#replace No negative and no positive
df["negative_review"] = df["negative_review"].apply(lambda x: str(x).replace("No Negative", "No"))
df["positive_review"] = df["positive_review"].apply(lambda x: str(x).replace("No Positive", "No"))

In [53]:
#function to check negative & positive duplicates
def check(x):
    pos, neg = x
    if pos ==  neg:
        return 1
    return 0

In [54]:
df['check_dup'] = [check(x) for x in df[['positive_review','negative_review']].values]
index_col= df[df['check_dup']==1].index

In [55]:
df[df['check_dup']==1][['positive_review','negative_review']].head()

,positive_review,negative_review
1403,No,No
2451,The hotel good location and clean but some st...,The hotel good location and clean but some st...
2872,Ok,Ok
3067,Standard Hotel,Standard Hotel
5839,I was completely disappointed and mad since t...,I was completely disappointed and mad since t...


In [56]:
for x in index_col:
    df['positive_review'][x]='nothing'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
#dedupe duplicates
df[df['check_dup']==1][['positive_review','negative_review']].head()

,positive_review,negative_review
1403,nothing,No
2451,nothing,The hotel good location and clean but some st...
2872,nothing,Ok
3067,nothing,Standard Hotel
5839,nothing,I was completely disappointed and mad since t...


In [60]:
#check if there is null
df.isnull().sum()

hotel_address                                 0
additional_number_of_scoring                  1
review_date                                   1
average_score                                 1
hotel_name                                    0
reviewer_nationality                          1
negative_review                               0
review_total_negative_word_counts             1
total_number_of_reviews                       1
positive_review                               0
review_total_positive_word_counts             1
total_number_of_reviews_reviewer_has_given    1
reviewer_score                                1
tags                                          1
days_since_review                             1
lat                                           1
lng                                           1
check_dup                                     0
dtype: int64

In [61]:
columns = ['additional_number_of_scoring', 'review_date', 'average_score','reviewer_nationality','review_total_negative_word_counts','total_number_of_reviews','review_total_positive_word_counts','total_number_of_reviews_reviewer_has_given','reviewer_score','days_since_review','check_dup']
df.drop(columns, inplace=True, axis=1)

In [62]:
df.isnull().sum()

hotel_address      0
hotel_name         0
negative_review    0
positive_review    0
tags               1
lat                1
lng                1
dtype: int64

In [63]:
df.dropna(subset=['lat','lng'],inplace=True)

In [64]:
geocode=pd.read_pickle('/content/geocode.pkl')

In [65]:
new_df= pd.merge(df,geocode,on='hotel_name',how='outer')

In [67]:
new_df.to_pickle('review.pkl')